# 1. Delta $\Delta$

`delta` $\Delta$ is the rate of change of the option price with respect to the price of the underlying asset.
So if $c$ is (call) option price and $S$ is stock price then 

$$\Delta=\frac{\partial c}{\partial S}$$

For European option, we have `delta` equals:

- Long position of European call option: $N(d_1)$
- Short position of European call option: $-N(d_{1})$
- European put option: $N(d_{1})-1$

where 

$$d_{1}=\frac{\ln(S_{0}/K)+(r+\sigma^{2}/2)T}{\sigma\sqrt{T}}$$

Let create a function for `d1` first:

In [1]:
def getD1(S0, K, r, sigma, T):
    import numpy as np
    return (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))

In [3]:
# example
getD1(49, 50, 0.05, 0.2, 20/52)

0.05418135192383571

To get $N(d1)$, we use the package `scipy`:

In [4]:
import scipy.stats as scs
scs.norm.cdf(d1(49, 50, 0.05, 0.2, 20/52))

0.5216046610663964

We can create a composite function, which includes `d1` function, to calculate `delta`:

- We should learn the idea of `scope` in programming
- We also need to check `if ... else if ... else if ...` to return the `delta` w.r.t. the corresponding option: read this [documentation](https://www.tutorialspoint.com/python/python_if_else.htm)

In [6]:
def getOptionDelta(S0, K, r, sigma, T, call=True, long=True):
    import scipy.stats as scs
    import numpy as np
    # Step 1: calculate d1
    def getD1(S0, K, r, sigma, T):
        return (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d1 = getD1(S0, K, r, sigma, T)
    # Step 2: return result
    if (call == True & long == True):
        return scs.norm.cdf(d1)
    elif (call == True & long == False):
        return -scs.norm.cdf(d1)
    else:
        return scs.norm.cdf(d1) - 1

In [7]:
getOptionDelta(49, 50, 0.05, 0.2, 20/52)

0.5216046610663964

In [8]:
getOptionDelta(49, 50, 0.05, 0.2, 20/52, long=False)

-0.47839533893360364

In [9]:
getOptionDelta(49, 50, 0.05, 0.2, 20/52, call=False)

-0.47839533893360364

# 2. Theta $\Theta$

The theta ($\Theta$) is the rate of change of the value of the portfolio with respect to the passage of time (like, when 1 day passes) with all else remaining the same.

To get the PDF of a number, we need:

In [11]:
import scipy.stats as scs
scs.norm.pdf(1.96)

0.058440944333451476

In [13]:
scs.norm.pdf(-1.64)

0.10396109532876424

OK, now let's go to get $\Theta$:

In [10]:
def getD2(S0, K, r, sigma, T):
    import numpy as np
    return (np.log(S0/K) + (r - sigma**2/2)*T)/(sigma*np.sqrt(T))

In [14]:
def getOptionTheta(S0, K, r, sigma, T, call=True):
    import scipy.stats as scs
    import numpy as np
    # Step 1: calculate d1 & d2
    def getD1(S0, K, r, sigma, T):
        return (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    def getD2(S0, K, r, sigma, T):
        return (np.log(S0/K) + (r - sigma**2/2)*T)/(sigma*np.sqrt(T))

    d1 = getD1(S0, K, r, sigma, T)
    d2 = getD2(S0, K, r, sigma, T)
    pdf1 = scs.norm.pdf(d1)
    pdf2 = scs.norm.pdf(d2)
    cdf2 = scs.norm.cdf(d2)
    cdf_2 = scs.norm.cdf(-d2)

    # Step 2: return result
    if (call == True):
        return ( -(S0*pdf1*sigma)/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*cdf2 )
    else:
        return ( -(S0*pdf1*sigma)/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*cdf_2 )

In [15]:
getOptionTheta(S0=50, K=49, r=0.05, sigma=0.2, T=20/52)

-4.436781777484395

# 3. Gamma $\Gamma$

The gamma ($\Gamma$) of a portfolio of options is the rate of change of the portfolio's `delta` with respect to the price of the underlying asset. It is the second partial derivative of the portfolio with respect to asset price:

$$\Gamma=\frac{\partial^{2}\Pi}{\partial S^{2}}$$

In [16]:
def getOptionGamma(S0, K, r, sigma, T, call=True):
    import scipy.stats as scs
    import numpy as np
    # Step 1: calculate d1
    def getD1(S0, K, r, sigma, T):
        return (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d1 = getD1(S0, K, r, sigma, T)
    # Step 2: return result
    return scs.norm.pdf(d1)/(S0*sigma*np.sqrt(T))


In [18]:
getOptionGamma(49, 50, 0.05, 0.2, 20/52)

0.06554403934784439

For other `Greeks`, we can calculate similarly.